# CRORIS Projekti API pt.2

Second part of data importing, after the API is called, we are now processing the raw response saved in a .csv

In [36]:
#imports
import pandas as pd
import json
import time
import datetime
import ast
from contextlib import suppress

drive_path = 'data2/'

In [37]:
# initiate dataframes
df_ustanova = pd.DataFrame(columns=['id','mbu','ustanovaNaziv','aaiDomain','created'])#done
df_osoba = pd.DataFrame(columns=['id','osobaPoirotId','ime','prezime','ustanovaId','created'])#done
df_projekt = pd.DataFrame(columns=['id','projektPoirotId','hrSifraProjekta','tipProjekta_id','acro',
                                   'pocetak','kraj','currencyCode','totalCost','title','summary','financijer_id','created'])#done
df_financijer = pd.DataFrame(columns=['id','financijerPoirotId','entityId','entityNameHr','entityNameEn',
                                      'vrstaIzvoraFinanciranjaId','vrstaIzvoraFinanciranjaName','pozivId','pozivNaziv','created'])#done
# relational dfs
df_osoba_projekt = pd.DataFrame(columns=['osoba_id','projekt_id','pocetak','kraj','projektnaUloga','created'])#done
df_projekt_financijer = pd.DataFrame(columns=['projekt_id','financijer_id','created'])#done
df_projekt_tip = pd.DataFrame(columns=['projekt_id','tipProjektaId','tipProjektaNaziv','created'])#done
df_projekt_ustanova = pd.DataFrame(columns=['projekt_id','ustanova_id','amount','currencyCode','klasifikacijaNaziv','created'])#done

In [38]:
# function for populating pandas dfs
def populate_dfs(response,df_ustanova,df_osoba,df_projekt,df_financijer,df_osoba_projekt,df_projekt_financijer,df_projekt_tip,df_projekt_ustanova):
    #initiate variables
    projekt_id,tipProjektaId,tipProjektaNaziv,projektPoirotId,hrSifraProjekta,tipProjekta_id,acro,pocetak,kraj,currencyCode,totalCost,title,summary,financijer_id = (None,)*14
    
    with suppress(KeyError): projekt_id = response['id']
    with suppress(KeyError): tipProjektaId = response['tipProjekta']['id']
    with suppress(KeyError): tipProjektaNaziv = response['tipProjekta']['naziv']
    with suppress(KeyError): projektPoirotId = response['projektPoirotId']   
    with suppress(KeyError): hrSifraProjekta = response['hrSifraProjekta']
    with suppress(KeyError): tipProjekta_id = response['tipProjekta']['id']
    with suppress(KeyError): acro = response['acro']
    with suppress(KeyError): pocetak = response['pocetak']
    with suppress(KeyError): kraj = response['kraj']
    with suppress(KeyError): currencyCode = response['currencyCode']
    with suppress(KeyError): totalCost = response['totalCost']
    with suppress(KeyError): title = response['title'][0]['naziv']
    with suppress(KeyError): summary = response['summary'][0]['naziv']
    with suppress(KeyError): financijer_id = response['financijerResources']['_embedded']['financijeri'][0]['id']
    with suppress(KeyError): created = datetime.datetime.now()

    try:
        response['ustanoveResources']
    except:
        print(projekt_id, '- Nema ustanove!')
    else:
        #iterate over USTANOVA and save as pandas df together with relation with projekt
        for ustanova in response['ustanoveResources']['_embedded']['ustanove']: 
            ustanova_id,mbu,ustanovaNaziv,aaiDomain,amount,currencyCode,klasifikacijaNaziv,ustanovaPocetak,ustanovaKraj = (None,)*9
            #ustanova
            with suppress(KeyError): ustanova_id = ustanova['id']
            with suppress(KeyError): mbu = ustanova['mbu']
            with suppress(KeyError): ustanovaNaziv = ustanova['naziv']
            with suppress(KeyError): aaiDomain = ustanova['aaiDomain']

            #projekt ustanova relation
            with suppress(KeyError): amount = ustanova['amount']
            with suppress(KeyError): currencyCode = ustanova['currencyCode']
            with suppress(KeyError): klasifikacijaNaziv = ustanova['klasifikacija']['naziv']
            with suppress(KeyError): ustanovaPocetak = ustanova['pocetak']
            with suppress(KeyError): ustanovaKraj = ustanova['kraj']
      
            #save each ustanova from projekt
            df_ustanova = pd.concat([df_ustanova, pd.DataFrame(data= {'id' :ustanova_id,
                                                        'mbu' :mbu,
                                                        'ustanovaNaziv' :ustanovaNaziv,
                                                        'aaiDomain' :aaiDomain,
                                                        'created' :created}, index=[0])])
            
            #save each relation between ustanova and projekt
            df_projekt_ustanova = pd.concat([df_projekt_ustanova, pd.DataFrame(data={'projekt_id' :projekt_id,
                                                              'ustanova_id' :ustanova_id,
                                                              'amount' :amount,
                                                              'currencyCode' :currencyCode,
                                                              'klasifikacijaNaziv' :klasifikacijaNaziv,
                                                              'pocetak' :ustanovaPocetak,
                                                              'kraj' :ustanovaKraj,
                                                              'created' :created}, index=[0])])

    try:
        response['osobeResources']
    except:
        print(projekt_id, '- Nema osobe!')
    else:
        #iterate over all OSOBA from projekt and save to pandas dfs
        for osoba in response['osobeResources']['_embedded']['osobe']:
            osobaPoirotId,persId,ime,prezime,projektnaUloga,ustanovaId,pocetak,kraj = (None,)*8
            with suppress(KeyError): osobaPoirotId = osoba['osobaPoirotId']
            with suppress(KeyError): persId = osoba['persId']
            with suppress(KeyError): ime = osoba['ime']
            with suppress(KeyError): prezime = osoba['prezime']
            with suppress(KeyError): projektnaUloga = osoba['klasifikacija']['naziv']
            with suppress(KeyError): ustanovaId = osoba['ustanovaId']
            with suppress(KeyError): pocetak = osoba['pocetak']
            with suppress(KeyError): kraj = osoba['kraj']

            #save each osoba from projekt
            df_osoba = pd.concat([df_osoba, pd.DataFrame(data= {'id' :persId,
                                                      'osobaPoirotId' :osobaPoirotId,
                                                      'ime' :ime,
                                                      'prezime' :prezime,
                                                      'ustanovaId' :ustanovaId,
                                                      'created' :created}, index=[0])])
            
            #save osoba projekt relation
            df_osoba_projekt = pd.concat([df_osoba_projekt, pd.DataFrame(data= {'osoba_id' :persId,
                                                              'projekt_id' :projekt_id,
                                                              'pocetak' :pocetak,
                                                              'kraj' :kraj,
                                                              'projektnaUloga' :projektnaUloga,
                                                              'created' :created}, index=[0])])

    #save PROJEKT details to pandas df
    df_projekt = pd.concat([df_projekt, pd.DataFrame(data={'id' :projekt_id,
                                              'projektPoirotId' :projektPoirotId,
                                              'hrSifraProjekta' :hrSifraProjekta,
                                              'tipProjekta_id' :tipProjekta_id,
                                              'acro' :acro,
                                              'pocetak' :pocetak,
                                              'kraj' :kraj,
                                              'currencyCode' :currencyCode,
                                              'totalCost' :totalCost,
                                              'title' :title,
                                              'summary' :summary,
                                              'financijer_id' :financijer_id,
                                              'created' :created}, index=[0])])
    
    #iterate over FINANCIJER and save as pandas df together with relation with projekt
    try:
        response['financijerResources']
    except:
        print(projekt_id, '- Nema financijera!')
    else:
        for financijer in response['financijerResources']['_embedded']['financijeri']:
            financijer_id,financijerPoirotId,entityId,entityNameHr,entityNameEn,vrstaIzvoraFinanciranjaId,vrstaIzvoraFinanciranjaName,pozivId,pozivNaziv=(None,)*9
            with suppress(KeyError): financijer_id = financijer['id']
            with suppress(KeyError): financijerPoirotId = financijer['financijerPoirotId']
            with suppress(KeyError): entityId = financijer['entityId']
            with suppress(KeyError): entityNameHr = financijer['entityNameHr']
            with suppress(KeyError): entityNameEn = financijer['entityNameEn']
            with suppress(KeyError): vrstaIzvoraFinanciranjaId = financijer['vrstaIzvoraFinanciranjaId']
            with suppress(KeyError): vrstaIzvoraFinanciranjaName = financijer['vrstaIzvoraFinanciranjaName']
            with suppress(KeyError): pozivId = financijer['pozivId'] 
            with suppress(KeyError): pozivNaziv = financijer['pozivNaziv']

            #save each financijer from projekt
            df_financijer = pd.concat([df_financijer, pd.DataFrame(data={'id' :financijer_id,
                                                      'financijerPoirotId' :financijerPoirotId,
                                                      'entityId' :entityId,
                                                      'entityNameHr' :entityNameHr,
                                                      'entityNameEn' :entityNameEn,
                                                      'vrstaIzvoraFinanciranjaId' :vrstaIzvoraFinanciranjaId,
                                                      'vrstaIzvoraFinanciranjaName' :vrstaIzvoraFinanciranjaName,
                                                      'pozivId' :pozivId,
                                                      'pozivNaziv' :pozivNaziv,
                                                      'created' :created}, index=[0])])
            
            #save each financijer from projekt
            df_projekt_financijer = pd.concat([df_projekt_financijer, pd.DataFrame(data={'projekt_id' :projekt_id,
                                                          'financijer_id' :financijer_id,
                                                          'created' :created}, index=[0])])

    #save each tip projekta
    df_projekt_tip = pd.concat([df_projekt_tip, pd.DataFrame(data={'projekt_id' :projekt_id,
                                                          'tipProjektaId' :tipProjektaId,
                                                          'tipProjektaNaziv' :tipProjektaNaziv,
                                                          'created' :created}, index=[0])])
    
    return df_ustanova,df_osoba,df_projekt,df_financijer,df_osoba_projekt,df_projekt_financijer,df_projekt_tip,df_projekt_ustanova

In [39]:
# read the scrapped API responses saved in csv to df
df_response = pd.read_csv(drive_path+'CRORIS_df_response_20220925-124622.csv', index_col=0) 

In [40]:
# MAIN
for index, row in df_response.iterrows():
    response = ast.literal_eval(row['response'])
    #print(response['id'])
    # call function populate_dfs() and assign returned dfs to our pandas dfs
    df_ustanova,df_osoba,df_projekt,df_financijer,df_osoba_projekt,df_projekt_financijer,df_projekt_tip,df_projekt_ustanova = populate_dfs(response,df_ustanova,df_osoba,df_projekt,df_financijer,df_osoba_projekt,df_projekt_financijer,df_projekt_tip,df_projekt_ustanova)
    
print("END")

526 - Nema financijera!
527 - Nema financijera!
528 - Nema financijera!
529 - Nema financijera!
530 - Nema financijera!
531 - Nema financijera!
532 - Nema financijera!
533 - Nema financijera!
534 - Nema financijera!
574 - Nema osobe!
609 - Nema osobe!
678 - Nema ustanove!
678 - Nema osobe!
711 - Nema ustanove!
711 - Nema osobe!
795 - Nema osobe!
2866 - Nema osobe!
3189 - Nema osobe!
3190 - Nema osobe!
3191 - Nema osobe!
3195 - Nema osobe!
3201 - Nema osobe!
3202 - Nema osobe!
3204 - Nema osobe!
3206 - Nema osobe!
3207 - Nema osobe!
3208 - Nema osobe!
3212 - Nema osobe!
3213 - Nema osobe!
3215 - Nema osobe!
3224 - Nema osobe!
3230 - Nema osobe!
3266 - Nema osobe!
3480 - Nema osobe!
3481 - Nema osobe!
3484 - Nema osobe!
3486 - Nema osobe!
3487 - Nema osobe!
3488 - Nema osobe!
3490 - Nema osobe!
3491 - Nema osobe!
3492 - Nema osobe!
3493 - Nema osobe!
3494 - Nema osobe!
3495 - Nema osobe!
3496 - Nema osobe!
3497 - Nema osobe!
3498 - Nema osobe!
3499 - Nema osobe!
3503 - Nema osobe!
3504 -

In [43]:
df_projekt_ustanova.head()

,projekt_id,ustanova_id,amount,currencyCode,klasifikacijaNaziv,created,pocetak,kraj
0,1,66,990253.36,HRK,nositelj,2022-09-27 01:08:54.118317,2015-09-01,2020-01-31
0,2,66,936100.00,HRK,nositelj,2022-09-27 01:08:54.126621,2015-09-01,2019-01-09
0,3,148,981988.00,HRK,nositelj,2022-09-27 01:08:54.136852,2015-10-01,2019-01-10
0,4,70,969952.00,HRK,nositelj,2022-09-27 01:08:54.141735,2015-09-01,2019-01-09
0,5,70,676134.00,HRK,nositelj,2022-09-27 01:08:54.146935,2015-07-01,2018-01-07


In [42]:
# time conversion in pandas
df_projekt['pocetak'] = pd.to_datetime(df_projekt['pocetak'], format = "%d.%m.%Y", errors = 'coerce')
df_projekt['kraj'] = pd.to_datetime(df_projekt['kraj'], format = "%d.%m.%Y", errors = 'coerce')
df_osoba_projekt['pocetak'] = pd.to_datetime(df_osoba_projekt['pocetak'], format = "%d.%m.%Y", errors = 'coerce')
df_osoba_projekt['kraj'] = pd.to_datetime(df_osoba_projekt['kraj'], format = "%d.%m.%Y", errors = 'coerce')
df_projekt_ustanova['pocetak'] = pd.to_datetime(df_projekt_ustanova['pocetak'], format = "%d.%m.%Y", errors = 'coerce')
df_projekt_ustanova['kraj'] = pd.to_datetime(df_projekt_ustanova['kraj'], format = "%d.%m.%Y", errors = 'coerce')

In [44]:
df_osoba_projekt.query('osoba_id == 38162') # greska kraj na 6667

,osoba_id,projekt_id,pocetak,kraj,projektnaUloga,created
0,38162,781,2014-05-01,2015-01-01,voditelj,2022-09-27 01:08:58.340438
0,38162,3635,2017-01-01,2018-12-31,voditelj na ustanovi,2022-09-27 01:09:15.021781
0,38162,3796,2018-06-01,2021-11-01,suradnik,2022-09-27 01:09:16.448271
0,38162,4370,2019-01-01,2019-12-31,voditelj na ustanovi,2022-09-27 01:09:22.604066
0,38162,4374,2020-01-01,2021-12-31,voditelj na ustanovi,2022-09-27 01:09:22.652436
0,38162,4838,2019-01-10,2022-12-31,voditelj,2022-09-27 01:09:30.313401
0,38162,5471,2019-10-01,2022-09-30,suradnik,2022-09-27 01:09:38.439380
0,38162,5472,2020-06-29,2022-12-29,voditelj,2022-09-27 01:09:38.519748
0,38162,5473,2020-09-01,2023-08-31,suradnik,2022-09-27 01:09:38.529833
0,38162,5475,2021-01-01,2023-12-31,suradnik,2022-09-27 01:09:38.567143


In [45]:
#drop duplicates from all pandas dfs
df_ustanova = df_ustanova.drop_duplicates(subset='id', keep='last')
df_osoba = df_osoba.drop_duplicates(subset='id', keep='last')
df_projekt = df_projekt.drop_duplicates(subset='id', keep='last')
df_financijer = df_financijer.drop_duplicates(subset='id', keep='last')
df_osoba_projekt = df_osoba_projekt.drop_duplicates(subset=['osoba_id','projekt_id'], keep='last')
df_projekt_financijer = df_projekt_financijer.drop_duplicates(subset=['projekt_id','financijer_id'],keep='last')
df_projekt_tip = df_projekt_tip.drop_duplicates(subset='projekt_id', keep='last')
df_projekt_ustanova = df_projekt_ustanova.drop_duplicates(subset=['projekt_id','ustanova_id'], keep='last')

In [46]:
df_ustanova.to_csv(drive_path+'CRORIS_df_ustanova.csv')
df_osoba.to_csv(drive_path+'CRORIS_df_osoba.csv')
df_projekt.to_csv(drive_path+'CRORIS_df_projekt.csv')
df_financijer.to_csv(drive_path+'CRORIS_df_financijer.csv')
df_osoba_projekt.to_csv(drive_path+'CRORIS_df_osoba_projekt.csv')
df_projekt_financijer.to_csv(drive_path+'CRORIS_df_projekt_financijer.csv')
df_projekt_tip.to_csv(drive_path+'CRORIS_df_projekt_tip.csv')
df_projekt_ustanova.to_csv(drive_path+'CRORIS_df_projekt_ustanova.csv')